# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откроем и изучим файл

Сначала импортируем все необходимые библиотеки.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from warnings import filterwarnings

Подгрузим датасет и изучим его.

In [2]:
df = pd.read_csv('C:\\Users\\admin\\Desktop\\DS_yandex\\05_Рекомендация тарифов\\users_behavior.csv')
display(df.head())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В подгруженном датасете содержится информация о месячной активности пользователей услуг: 
- **calls** - количество звонков;
- **minutes** - использованные минуты телефонных разговоров;
- **messages** - отправленные смс;
- **mb_used** - использованный интернет-траффик (мб);
- **is_ultra** - признак, показывающий является ли пользователь абонентом тарифа "Ultra" (значение "1") или нет (значение "0").

В качестве целевого призника выберем столбец **is_ultra**.

## Разобьем данные на выборки

Разобьем данные на обучающую, валидационную и тестовую выборки.

In [3]:
features = df.drop('is_ultra', axis = 1) 
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print('Размер обучающей выборки', features_train.shape[0], 'строк.\nРазмер валидационной выборки', 
      features_valid.shape[0], 'строк.\nРазмер тестовой выборки', features_test.shape[0], 'строк.')

Размер обучающей выборки 1928 строк.
Размер валидационной выборки 643 строк.
Размер тестовой выборки 643 строк.


## Исследуем модели

Обучим несколько моделей и проверим их.

### Дерево решений

In [4]:
%%time
model = DecisionTreeClassifier(random_state=12345)
parametrs = { 'max_depth': range (1,11),
              'min_samples_leaf': range (1,11),
              'min_samples_split': range (2,11) }
decisionTree_model = GridSearchCV(model, parametrs, n_jobs=-1, cv=5)
decisionTree_model.fit(features_train, target_train)

CPU times: total: 2.38 s
Wall time: 7.17 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'min_samples_split': range(2, 11)})

In [5]:
print('Best_score лучшей модели decisionTree -',decisionTree_model.best_score_)
print('\nПараметры оптимальной модели:\t max_depth \t   -', decisionTree_model.best_params_['max_depth'],
     '\n\t\t\t\t min_samples_leaf  -', decisionTree_model.best_params_['min_samples_leaf'],
     '\n\t\t\t\t min_samples_split -', decisionTree_model.best_params_['min_samples_split'])

Best_score лучшей модели decisionTree - 0.8065365722360539

Параметры оптимальной модели:	 max_depth 	   - 7 
				 min_samples_leaf  - 10 
				 min_samples_split - 2


### Случайный лес

In [6]:
%%time
model = RandomForestClassifier(n_jobs=-1, random_state=12345)
parametrs = { 'n_estimators': range (1, 51, 5),
              'max_depth': range (1,11),
              'min_samples_leaf': range (1,11) }
randomForest_model = GridSearchCV(model, parametrs, cv=5)
randomForest_model.fit(features_train, target_train)

CPU times: total: 3min 57s
Wall time: 3min 50s


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=12345),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'n_estimators': range(1, 51, 5)})

In [7]:
print('Best_score лучшей модели randomForest -',randomForest_model.best_score_)
print('\nПараметры оптимальной модели:\t n_estimators\t  -', randomForest_model.best_params_['n_estimators'],
     '\n\t\t\t\t max_depth\t  -', randomForest_model.best_params_['max_depth'],
     '\n\t\t\t\t min_samples_leaf -', randomForest_model.best_params_['min_samples_leaf'])

Best_score лучшей модели randomForest - 0.8241746854182086

Параметры оптимальной модели:	 n_estimators	  - 16 
				 max_depth	  - 10 
				 min_samples_leaf - 4


### Логистическая регрессия

In [8]:
%%time
filterwarnings('ignore')
model = LogisticRegression(n_jobs=-1, random_state=12345)
parametrs = { 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'] }
logisticRegression_model = GridSearchCV(model, parametrs, cv=5)
logisticRegression_model.fit(features_train, target_train)

CPU times: total: 219 ms
Wall time: 1.08 s


GridSearchCV(cv=5, estimator=LogisticRegression(n_jobs=-1, random_state=12345),
             param_grid={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [9]:
print('Best_score лучшей модели logisticRegression -',logisticRegression_model.best_score_)
print('\nПараметры оптимальной модели:\t solver -', logisticRegression_model.best_params_['solver'])

Best_score лучшей модели logisticRegression - 0.7484529977794226

Параметры оптимальной модели:	 solver - newton-cg


### Метод k-ближайших соседей (классификация) 

In [10]:
%%time
model = KNeighborsClassifier()
parametrs = { 'n_neighbors': range (1,101) }
kneighbors_model = GridSearchCV(model, parametrs, cv=5)
kneighbors_model.fit(features_train, target_train)

CPU times: total: 11.4 s
Wall time: 11.3 s


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 101)})

In [11]:
print('Best_score лучшей модели kneighbors -',kneighbors_model.best_score_)
print('\nПараметры оптимальной модели:\t n_neighbors -', kneighbors_model.best_params_['n_neighbors'])

Best_score лучшей модели kneighbors - 0.767645515106655

Параметры оптимальной модели:	 n_neighbors - 15


## Проверим модель на тестовой выборке

Проверим наши модели на тестовой выборке и сведем все в одну таблицу.

In [12]:
ml = [decisionTree_model, randomForest_model, logisticRegression_model, kneighbors_model]
mln = ['DecisionTree', 'RandomForest', 'LogisticRegression', 'KNeighbors']
acc_val, acc_test = [], []
for i in ml: 
    acc_test.append(accuracy_score(target_test, i.predict(features_test)))
df_a = pd.DataFrame()
df_a['model'] = mln
df_a['accuracy_test'] = acc_test
display(df_a.sort_values(by = 'accuracy_test', ascending = False))

,model,accuracy_test
1,RandomForest,0.802488
0,DecisionTree,0.788491
3,KNeighbors,0.746501
2,LogisticRegression,0.738725


### Вывод

Сравнивая данные по моделям можно сказать, что для данного набора данных эффективность моделей ранжируется следующим образом:
1. **RandomForest_model**;
2. **DecisionTree_model**;
3. **KNeighbors_model**;
4. **LogisticRegression_model**.

Наиболее перспективными выглядят модели **Дерево решений** и **Случайный лес**, остановим свой выбор на втором, так как он показал лучшие результаты.

## (бонус) Проверим модели на адекватность

Проверим выбранную модель на адекватность. Для этого воспользуемся DummyClassifier.

In [13]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train, target_train)
dummy_model.score(features_test, target_test)

0.6842923794712286

Все созданные нами модели(**Случайный лес** в особенности) показывают более высокие результаты, нежели те, которые выдала модель **dummy_model**. Соответственно, проверка на адекватность пройдена.